# Differentiable Physics VII: Incompressible Navier Stokes

## Navier Stokes Equations

We limit the scope of the solver with the following constraints:

- We use the conservative form of the equations, meaning we are looking at a volume which is fixed in space and the fluid is moving through it.
- We use a 2D grid based approach, where each cell in the grid represents a volume element and has size $\Delta x \times \Delta y$.

The following quantities are defined on the grid:

- $\vec{v_{i,j}}$ : velocity vector field with x / y components $u$ and $v$
- $\rho$ : pressure scalar field
- $E$ : Energy scalar field

We can formulate the compressible navier stokes equations in 2D as:

$$\frac{U}{\Delta t} + \frac{E}{\Delta x} + \frac{F}{\Delta y} = 0$$

where we have:

$$ U = \begin{bmatrix} \rho \\ \rho u \\ \rho v \\ E \end{bmatrix} \quad
E = \begin{bmatrix} \rho u \\ \rho u^2 + p - \tau_{xx} \\ \rho u v - \tau_{xy} \\ (E + p) u - u \tau_{xx} - v \tau_{xy} + q_x \end{bmatrix} 
F = \begin{bmatrix} \rho v \\ \rho u v - \tau_{xy} \\ \rho v^2 + p - \tau_{yy} \\ (E + p) v - u \tau_{xy} - v \tau_{yy} + q_y \end{bmatrix} $$

where $\tau_{xx}$, $\tau_{xy}$, $\tau_{yy}$ are the components of the viscous stress tensor and $q_x$, $q_y$ are the components of the heat flux vector.

$$ \tau_{xx} = \mu \left( 2 \frac{\partial u}{\partial x} - \frac{2}{3} \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \right) \right) \quad
\tau_{xy} = \mu \left( \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x} \right) \quad
\tau_{yy} = \mu \left( 2 \frac{\partial v}{\partial y} - \frac{2}{3} \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \right) \right) $$

$$ q_x = - \lambda \frac{\partial T}{\partial x} \quad
q_y = - \lambda \frac{\partial T}{\partial y} $$

where $\mu$ is the dynamic viscosity and $\lambda$ is the thermal conductivity.


We now insert the navier stokes equations into the mac cormack method. We start with the predictor step:

$$\bar{U}_{i,j}^{n+1} = U_{i,j}^n - \frac{\Delta t}{\Delta x} \left( E_{i+1, j}^{n} - E_{i, j}^{n} \right) - \frac{\Delta t}{\Delta y} \left( F_{i, j+1}^{n} - F_{i, j}^{n} \right)$$

and the corrector step:

$$U_{i,j}^{n+1} = \frac{1}{2} \left[ \left( U_{i,j}^n + \bar{U}_{i,j}^{n+1} \right) - \frac{\Delta t}{ \Delta x} \left( \bar{E}_{i, j}^{n+1} - \bar{E}_{i-1, j}^{n+1} \right) - \frac{\Delta t}{\Delta y} \left( \bar{F}_{i, j}^{n+1} - \bar{F}_{i, j-1}^{n+1} \right) \right]$$

In [1]:
import jax.numpy as jnp
from jax import value_and_grad, jit
from functools import partial

def fwd_diff(f : jnp.ndarray, dx : float, axis : int):
    return jnp.roll(f, -1, axis=axis) - f / dx

def bwd_diff(f : jnp.ndarray, dx : float, axis : int):
    return f - jnp.roll(f, 1, axis=axis) / dx

def ctr_diff(f : jnp.ndarray, dx : float, axis : int):
    return (jnp.roll(f, -1, axis=axis) - jnp.roll(f, 1, axis=0)) / (2*dx)

In [ ]:
# U : [rho, rho*u, rho*v, E_total]
def mac_cormack_2D(
    U : jnp.ndarray, 
    dt : float, 
    dx : float, 
    dy : float, 
    mu : float, # viscosity
    c_v : float, # specific heat constant volume
    c_p : float, # specific heat constant pressure
    R : float, # gas constant
    k : float, # thermal conductivity
):
    gamma = c_p / c_v
    rho = U[0]
    u = U[1]/rho
    v = U[2]/rho
    e = U[3]/U[1] - 0.5*(u**2 + v**2)

    T = (gamma - 1)*e / R
    p = (gamma - 1)*rho*e

    tau_xx = 2/3*mu*(2*fwd_diff(u, dx, axis=0) - fwd_diff(v, dy, axis=1))
    tau_yy = 2/3*mu*(2*fwd_diff(v, dy, axis=1) - fwd_diff(u, dx, axis=0))
    tau_xy = mu*(fwd_diff(u, dx, axis=0) + fwd_diff(v, dy, axis=1))

    q_x = -k*fwd_diff(T, dx, axis=0)
    q_y = -k*fwd_diff(T, dy, axis=1)

    